In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src import read_nz_file, read_jg_file, update_meta_data
from sklearn.model_selection import train_test_split

processed data - intermediate step

> raw data

> clean data

> preprocess data : store it in DB (better compared to file format) 

    - data lake
    
    or
    
    - DB model based SQL (Nice to have but not required if we deceide to save the data as a Feather file)

> 

## 1. Load and clean raw data

files from SensorLog iOS app has in total over 70 colums and precision of 12 decimal figures. The output file is over 135 MB, which is too large for GitHub. GitHub restricts the file size, therefore these files from SensorLog must be cleaned.

Below summary of steps which is done only for iOS files:

- read raw data as csv files
- remove unnecessary columns (captured in list 'remove_cols' below)
- round to 6 decimal places to reduce the size of files
- output dataframe as csv
- upload the csv on GitHub

# 2. Meta data

The data on different data files is captured in meta dataframe below:
- file name
- user (nz or jg)
- activity (running/cycling/walking/sitting)
- pocket (in which pocket handy was during the activity)
- position_x
- position_y


# 3. View data

Sensor activity data is captured from 2 different Apps:
- SensorLog (iOS) by user 'nz'
- AndrioSensor (Andriod) by user 'jg'

**Response**: 'Acivity' with 4 classes: running/walking/cycling/sitting



In [ ]:
update_meta_data()

In [ ]:
meta = pd.read_csv('data/meta.csv')
meta

In [ ]:
meta = meta[23:]

In [ ]:
all_train = pd.DataFrame()
all_test = pd.DataFrame()

def split_df(df: pd.DataFrame, hz: float = 100, test_proportion: float = 0.2):
    # select every nth row
    if test_proportion > 100:
        raise Exception('hz must be less than 100!')
    nth = int(100 / hz)
    df = df.iloc[::nth, :]

    # split into train and test
    if test_proportion >= 1 or test_proportion <= 0:
        raise Exception('test_proportion must be between 0 and 1!')

    my_train_files = []
    my_test_files = []

    # test_per_minute = int(60 * test_proportion * hz)
    # train_per_minute = int((60 * hz) - test_per_minute)

    while len(df) / hz > 0:
        if len(df) / hz < 600:
            x = df
            df = pd.DataFrame()
        else:
            x = df[:300 * hz:]
            df = df[300 * hz:]

        train_count  = int(len(x) * (1 - test_proportion))

        my_train_files.append(x.iloc[: train_count])
        my_test_files.append(x.iloc[train_count:])

    print([len(i) for i in my_train_files])
    print([len(i) for i in my_test_files])

    return (train, test)


for file, user, activity in zip(meta['file'], meta['user'], meta['activity']):
    if user == 'nz':
        df = read_nz_file(file, activity)

    elif user == 'jg':
        df = read_jg_file(file, activity)

    print(file, user, activity, df.shape)

    # split into train-test
    train, test = split_df(df, hz = 20, test_proportion = 0.2)

    # append to train and test

# X - y split for test data

In [ ]:
start = test['datetime'].min()
end = test['datetime'].max()

diff = end - start
# Get the interval in milliseconds
diff_in_milli_secs = diff.total_seconds() * 1000

diff_in_milli_secs